In [39]:
from sympy import symbols, Eq,solve, symbol

def isZero(grid,row,col):
    if(grid[row][col] == '0'):
        return True
    return False

def isUnknown(grid,row,col):
    if(grid[row][col] == ' '):
        return True
    return False

def isNumberGTZ(grid,row,col):
    if(not isZero(grid,row,col) and not isUnknown(grid,row,col)):
        return True
    return False

def buildEquation(grid,row,col,gsize):
    
    cellNum = int(grid[row][col]) #number currently in cell, will be greater than 0
    eq = -cellNum # Start the equation by moving the number to the left side of equation
    variables = () #Make an empty tuple to store variables, list will not work on objects of type "Symbols"
    rangeOfBoard  = range(0,gsize) #Range of minesweeper play area
    
    '''
    Look in surrounding area if in range of play area
    | | | |
    | |3| |
    | | | |
    '''
    for i in range(3):
        for j in range(3):
            currRow = row - 1 + i #Marks current row we are looking into 
            currCol = col - 1 + j #Marks current col we are looking into
            
            ij = (currRow,currCol) #Makes a tuple of the current position
            
            #If not the postion of the main cell
            if(ij != (row,col)):
                
                #If both current row and col are in the play area
                if(currRow in rangeOfBoard and currCol in rangeOfBoard):
                    
                    #if the current row and col represent an unexplored cell
                    if(isUnknown(grid,currRow,currCol)):
                        variable = symbols('('+str(currRow)+'|'+str(currCol)+')') #Make a variable named "g(row)|(col)"
                        variables = variables + (variable,) #Add variable to tuple
                        eq = eq + variable #Add variable to the equation
                        
    #[Equation,(variable,varaiable,..),cellNum]
    return [eq,variables,cellNum] 
    

def createEquations(grid,gsize):
    
    #Empty tuple to store individual equation infomation
    equationInfos = ()
    
    #Look into every cell
    for i in range(gsize):
        for j in range(gsize):
            
            #If the cell has a number greater than 0
            if(isNumberGTZ(grid,i,j)):
                #Make an equation and add it to the tuple
                equationInfos  = equationInfos + (buildEquation(grid,i,j,gsize),)
    
    
    return equationInfos

def solveEq(equationInfos):
    
    equations = ()
    syms = ()
    
    #Seperate equations and variables into two tuples
    for i in range(len(equationInfos)):
        equations = equations + (equationInfos[i][0],)
        
        for j in range(len(equationInfos[i][1])):
            if (equationInfos[i][1][j] not in syms):
                syms = syms + (equationInfos[i][1][j],)
    
    #Return dict of solved equations
    return solve(equations,syms)

def getRankedVars(equationInfos):
    
    variables = {}
    
    for i in range(len(equationInfos)):
        
        cellNum = equationInfos[i][2]
        
        for j in range(len(equationInfos[i][1])):
            
            variable = equationInfos[i][1][j].name
            
            if (variable not in variables):
                variables.update({variable:(cellNum,1)})
            else:
                variables.update({variable:(variables[variable][0]+cellNum, variables[variable][1]+1)})
                
    return variables

def strToCoord(strCoord):
    try:
        x = strCoord.replace('(','').replace(')','').split('|')
        return (int(x[0]),int(x[1]))
    except:
        return None


In [40]:
from IPython.display import clear_output

def solveMineSweeper(sweeper,size,mines):
    
    
    grid = sweeper.checkcell((0,0))
    clearedCells = []
    flags = []
    
    while(not sweeper.isfail() and len(sweeper.flags) != mines):
    
        #Get the equations created information [(equation,variables,int),...]
        eqInfo = createEquations(grid,size)
        
        #Get the solutions to the system of equations
        possibleSol = list(solveEq(eqInfo).items())
        
        #Get a dictionary of ranked cells
        rankedVars = getRankedVars(eqInfo)
        
        
        #Counter for new information
        newInfo = 0
        
        #Iterate through the potential solutions
        for sol in possibleSol:
            
            #Coords for possible solution
            solCoords = strToCoord(sol[0].name)
            
            
            #If there is no definitative solution
            if(solCoords != None):
                
                
                #if the solution is clear and not already accounted for
                if(sol[1] == 0 and solCoords not in clearedCells):

                    #Account for it
                    clearedCells.append(solCoords)

                    #Traverse it
                    grid = sweeper.checkcell(solCoords)

                    newInfo = newInfo + 1

                #if the solution is a bomb and not already accounted for
                elif(sol[1] == 1 and solCoords not in sweeper.flags):

                    #Place a flag on it
                    sweeper.flags.append(solCoords)

                    newInfo = newInfo + 1
                    
        if(newInfo == 0):
            #Choose the lowest score and checkcell
            rankedVars = sorted(rankedVars.items(), key=lambda x: (x[1][0]/x[1][1]), reverse=False)
            
            for i in rankedVars:
                
                #Coords for possible solution
                solCoords = strToCoord(i[0])
                
                if(i[1][1] != 1 and solCoords not in sweeper.flags):
            
                    #Account for it
                    clearedCells.append(solCoords)

                    #Traverse it
                    grid = sweeper.checkcell(solCoords)
                    
                    break
                
        clear_output(wait=True)
        sweeper.showcurrent()
        print("Mines found: "+str(len(sweeper.flags)))
        print("New info: "+str(newInfo))
           


In [47]:
import mines
import time

solved  = False

while(not solved):
    gridsize = 16
    n_mines = 40
    sweeper = mines.Mines(gridsize, n_mines)
    sweeper.showcurrent()

    start = time.time()
    solveMineSweeper(sweeper,gridsize,n_mines)
    end = time.time()
    
    solved = sweeper.checkmines()
    print("Total time: "+ str(end-start))
    print("Check Mines: "+str(solved))


     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  
   -----------------------------------------------------------------
 0 | 0 | 0 | 1 |   |   |   |   | 3 |   |   | 1 | 0 | 0 | 0 | 1 | 1 |
   -----------------------------------------------------------------
 1 | 0 | 0 | 2 | 2 |   | 4 |   | 3 | 3 | 3 | 3 | 1 | 1 | 0 | 1 |   |
   -----------------------------------------------------------------
 2 | 0 | 0 | 1 |   |   |   | 2 | 1 | 1 |   | 2 |   | 1 | 1 | 2 | 2 |
   -----------------------------------------------------------------
 3 | 0 | 1 | 2 | 2 | 2 | 1 | 1 | 0 | 1 | 1 | 2 | 1 | 1 | 1 |   | 1 |
   -----------------------------------------------------------------
 4 | 0 | 1 |   | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 |
   -----------------------------------------------------------------
 5 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 |
   -----------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 